In [ ]:
######################################################################################################
  ############ Creating TF-IDFs and making related article lists across allofplos ############ 
######################################################################################################

#####################################
 ##### The below opens / reads all of the bag_of_words csvs from a folder into Python.
#####################################
import os
import pandas as pd
path = "final_allofplos_bagofwords/"
fileList = os.listdir(path)
bag_of_words = {}
dois = []
old_doi_list = []
for i in range(len(fileList)):
    bag_of_words[i] = pd.read_csv("final_allofplos_bagofwords/"+fileList[i])
    dois = bag_of_words[i].iloc[:,0]
    bag_of_words[i].index = dois
    old_doi_list.extend(dois)




In [ ]:
############################################################ 
 ##### Use column sums of each dataframe to get final TF-IDF:
############################################################ 

###### Take out columns with strings in the values (would only be in first few columns but I went to 100 just in case):
  ## The index also has the DOI name so use index. 
for i in range(len(bag_of_words)):
    for j in range(100):
        if type(bag_of_words[i].iloc[1,j]) is str:
            bag_of_words[i] = bag_of_words[i].drop(bag_of_words[i].columns[j], axis=1)



#####################################
 ##### 1 Get TF part of TF-IDF (values divided by their row sums)
for i in range(len(bag_of_words)):
    row_sum = bag_of_words[i].sum(axis=1)
    bag_of_words[i] = bag_of_words[i] / row_sum[:,None]


    

#####################################
 ##### 2. Get IDF for all dataframes. 
##### Get IDF (column sums / sum of all words):
full_col_sum = pd.read_csv("Full_allofplos_BagOfWords.csv")

### Take out columns with the name 'Unnamed: X' (would only be in first column but I went up to 100 just in case):
import re
bad_col = re.compile("Unnamed:*")
for j in range(100):
    if len(bad_col.findall(full_col_sum.columns[j])) > 0:
        full_col_sum = full_col_sum.drop(full_col_sum.columns[j], axis=1)

### Transpose, get values, then sum.
sum_transposed = full_col_sum.T
sum_all_words = int(sum(sum_transposed.values))

idf = full_col_sum / sum_all_words




#####################################
 ##### 3. Get TF-IDF for all dataframes: 
####### Divide all 'bag_of_words' (which now are each value divided by it's row sum) by it's column's IDF:
#####################################
for i in range(len(bag_of_words)):
    ## Left join bag_of_words onto the full list of words: The makes the final row the corpus word count:
    bag_of_words[i] = bag_of_words[i].T
    trans_idf = idf.T
    bag_of_words[i] = bag_of_words[i].join(trans_idf).T
    ## Get TF-IDF for all rows; except final row (which is the sum of words in this loop)
       ## NOTE: This loop went quicker than the non-loop option: bag_of_words[i] = bag_of_words[i] / idf
    for j in range(len(bag_of_words[i].columns)):
        bag_of_words[i].iloc[:,j] = bag_of_words[i].iloc[:,j] / bag_of_words[i].iloc[(len(bag_of_words[i].index)-1),j]
    ## Drop the row with column sums:
    bag_of_words[i].drop(bag_of_words[i].tail(1).index,inplace=True)




In [ ]:
###### Combined with below, this allows you to time how fast your code runs:
import time
start_time = time.time()

import pandas as pd


##########################################
 ## MAIN: Get top 19 DOIS by TF-IDF correlation between DOI being compared and all other DOIs in corpus.
##########################################
##########################################
 ## Notes on time:
    ## Takes around 6 seconds per DOI (meaning the whole corpus takes a long time).
    ## You can break the DOIs to compare into chunks and do the program simultaneously across different computers.
    ## Also, it's set up to allow for starting and stopping: See the top of the loop.
##########################################


recommendations = 20

final_data = pd.DataFrame(columns = range(recommendations))

######################### I STOPPED AT THE BELOW (CAN JUST START IT UP, IT ACTUALLY STARTS AT THE FILE CALLED '12' BECAUSE IT GOES 0,1,10,11,12, ETC)

for iter_dataset in range(0, len(bag_of_words)-1):     ############ If you need to stop and start, change this line's range accordingly (e.g. if you went up to only chunk 15, you can start the program again starting at range(15,len(bag_of_words)-1)
    for iter_doi in range(len(bag_of_words[iter_dataset])):
        aa = pd.DataFrame(bag_of_words[iter_dataset].iloc[iter_doi,:].T)
        # Reduce DOI to compare data to words that exist in that DOI:
        aa = aa.fillna(0)
        aa = aa[aa.values > 0]
        #Get dataframe of only TF-IDFs in DOI being compared over 20. 
        x =  aa[aa.values > 20]
        corrs = pd.DataFrame()
        # This loop gets the correlations of the DOI for each bag_of_words:
        for i in range(0, (len(bag_of_words)-1)):
            new = bag_of_words[i]
            # If 'new' is the same dataframe as the one that has the DOI being compared, remove the DOI being compared from 'new'
            if new.index[0] == bag_of_words[iter_dataset].index[0]:
                new = new.drop([aa.columns[0]], axis = 0)
            #To increase speed, remove DOIs who have a summed TF-IDF of under 100 for TF-IDF over 5 in the DOI being compared.
            b = x.join(new.T)
            c = b.sum(axis=0)
            c = list(c[c.values < 1000].index)
            new = new.drop(c, axis=0)
            b = pd.concat([new, aa.T], sort=False)
            b = b.fillna(0).T
            #Do a corelation of the first dataframe with the second:
            c = pd.DataFrame(b[b.columns[0:]].corr()[b.columns[len(b.columns)-1]])[:-1]
            #Cut down to the number of recommendation and format to prepare for final data:
            corrs = pd.concat([corrs, c], sort=False)
            corrs = corrs.sort_values(corrs.columns[0], ascending=False).head(n = recommendations)
        to_compare = [corrs.columns[0]]
        corrs = pd.DataFrame(corrs.index)
        corrs.columns = to_compare
        corrs = corrs.T
        final_data = pd.concat([final_data, corrs], sort=False)
        ## NOTE: I put this in the loop becasue this takes so long, enables starting and stopping.
        final_data.to_csv(rf"RelatedDoiList_Best_Correlations_ALL.csv")    ############ If you stop and start, use a different file new (or else you'll overwrite the last one with an empty dataframe)

final_data.index = pd.Index(final_data.index, name = 'DOI Being Compared')   # use if index is the column of the compared DOI



In [ ]:

###############################################################
 ###### See file 'Update_related_article_list' for updating related article lists.
###############################################################

